For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
# Set up data science environment.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import spacy
import scipy
import seaborn as sns
import re
from nltk.corpus import inaugural, stopwords
from collections import Counter
%matplotlib inline

In [34]:
# Utility function to clean text.
def text_cleaner(text):
    text = re.sub(r'--', ' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub("[\<].*?[\>]", "", text)
    text = ' '.join(text.split())
    text = re.sub(r'\\n', ' ', text)

    return text[0:900000]


# View text lists.
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

In [35]:
# Import first inaugural address for each of ten Presidents.
washington = inaugural.raw("1789-Washington.txt")
jefferson = inaugural.raw("1801-Jefferson.txt")
lincoln = inaugural.raw("1861-Lincoln.txt")
roosevelt2 = inaugural.raw("1933-Roosevelt.txt")
eisenhower = inaugural.raw("1953-Eisenhower.txt")
kennedy = inaugural.raw("1961-Kennedy.txt")
reagan = inaugural.raw("1981-Reagan.txt")
bush = inaugural.raw("1989-Bush.txt")
clinton = inaugural.raw("1993-Clinton.txt")
obama = inaugural.raw("2009-Obama.txt")

In [4]:
# Load and clean data.
washington_clean = text_cleaner(washington)
jefferson_clean = text_cleaner(jefferson)
lincoln_clean = text_cleaner(jefferson)
roosevelt2_clean = text_cleaner(roosevelt2)
eisenhower_clean = text_cleaner(eisenhower)
kennedy_clean = text_cleaner(kennedy)
reagan_clean = text_cleaner(reagan)
bush_clean = text_cleaner(bush)
clinton_clean = text_cleaner(clinton)
obama_clean = text_cleaner(obama)

In [5]:
# Print the first 100 characters of each address.
presidents = [washington_clean,
              jefferson_clean,
              lincoln_clean,
              roosevelt2_clean,
              eisenhower_clean,
              kennedy_clean,
              reagan_clean,
              bush_clean,
              clinton_clean,
              obama_clean]

washington_clean[:1000]

'Fellow-Citizens of the Senate and of the House of Representatives: Among the vicissitudes incident to life no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the 14th day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time. On the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualifications, could not but o

In [6]:
jefferson_clean[:1000]

'Friends and Fellow Citizens: Called upon to undertake the duties of the first executive office of our country, I avail myself of the presence of that portion of my fellow citizens which is here assembled to express my grateful thanks for the favor with which they have been pleased to look toward me, to declare a sincere consciousness that the task is above my talents, and that I approach it with those anxious and awful presentiments which the greatness of the charge and the weakness of my powers so justly inspire. A rising nation, spread over a wide and fruitful land, traversing all the seas with the rich productions of their industry, engaged in commerce with nations who feel power and forget right, advancing rapidly to destinies beyond the reach of mortal eye when I contemplate these transcendent objects, and see the honor, the happiness, and the hopes of this beloved country committed to the issue, and the auspices of this day, I shrink from the contemplation, and humble myself bef

In [7]:
lincoln_clean[:1000]

'Friends and Fellow Citizens: Called upon to undertake the duties of the first executive office of our country, I avail myself of the presence of that portion of my fellow citizens which is here assembled to express my grateful thanks for the favor with which they have been pleased to look toward me, to declare a sincere consciousness that the task is above my talents, and that I approach it with those anxious and awful presentiments which the greatness of the charge and the weakness of my powers so justly inspire. A rising nation, spread over a wide and fruitful land, traversing all the seas with the rich productions of their industry, engaged in commerce with nations who feel power and forget right, advancing rapidly to destinies beyond the reach of mortal eye when I contemplate these transcendent objects, and see the honor, the happiness, and the hopes of this beloved country committed to the issue, and the auspices of this day, I shrink from the contemplation, and humble myself bef

In [8]:
roosevelt2_clean[:1000]

'I am certain that my fellow Americans expect that on my induction into the Presidency I will address them with a candor and a decision which the present situation of our Nation impels. This is preeminently the time to speak the truth, the whole truth, frankly and boldly. Nor need we shrink from honestly facing conditions in our country today. This great Nation will endure as it has endured, will revive and will prosper. So, first of all, let me assert my firm belief that the only thing we have to fear is fear itself nameless, unreasoning, unjustified terror which paralyzes needed efforts to convert retreat into advance. In every dark hour of our national life a leadership of frankness and vigor has met with that understanding and support of the people themselves which is essential to victory. I am convinced that you will again give that support to leadership in these critical days. In such a spirit on my part and on yours we face our common difficulties. They concern, thank God, only 

In [9]:
eisenhower_clean[:1000]

'My friends, before I begin the expression of those thoughts that I deem appropriate to this moment, would you permit me the privilege of uttering a little private prayer of my own. And I ask that you bow your heads: Almighty God, as we stand here at this moment my future associates in the executive branch of government join me in beseeching that Thou will make full and complete our dedication to the service of the people in this throng, and their fellow citizens everywhere. Give us, we pray, the power to discern clearly right from wrong, and allow all our words and actions to be governed thereby, and by the laws of this land. Especially we pray that our concern shall be for all the people regardless of station, race, or calling. May cooperation be permitted and be the mutual aim of those who, under the concepts of our Constitution, hold to differing political faiths; so that all may work for the good of our beloved country and Thy glory. Amen. My fellow citizens: The world and we have

In [10]:
kennedy_clean[:1000]

'Vice President Johnson, Mr. Speaker, Mr. Chief Justice, President Eisenhower, Vice President Nixon, President Truman, reverend clergy, fellow citizens, we observe today not a victory of party, but a celebration of freedom symbolizing an end, as well as a beginning signifying renewal, as well as change. For I have sworn I before you and Almighty God the same solemn oath our forebears l prescribed nearly a century and three quarters ago. The world is very different now. For man holds in his mortal hands the power to abolish all forms of human poverty and all forms of human life. And yet the same revolutionary beliefs for which our forebears fought are still at issue around the globe the belief that the rights of man come not from the generosity of the state, but from the hand of God. We dare not forget today that we are the heirs of that first revolution. Let the word go forth from this time and place, to friend and foe alike, that the torch has been passed to a new generation of Americ

In [11]:
reagan_clean[:1000]

"Senator Hatfield, Mr. Chief Justice, Mr. President, Vice President Bush, Vice President Mondale, Senator Baker, Speaker O'Neill, Reverend Moomaw, and my fellow citizens: To a few of us here today, this is a solemn and most momentous occasion; and yet, in the history of our Nation, it is a commonplace occurrence. The orderly transfer of authority as called for in the Constitution routinely takes place as it has for almost two centuries and few of us stop to think how unique we really are. In the eyes of many in the world, this every-4-year ceremony we accept as normal is nothing less than a miracle. Mr. President, I want our fellow citizens to know how much you did to carry on this tradition. By your gracious cooperation in the transition process, you have shown a watching world that we are a united people pledged to maintaining a political system which guarantees individual liberty to a greater degree than any other, and I thank you and your people for all your help in maintaining the

In [12]:
bush_clean[:1000]

"Mr. Chief Justice, Mr. President, Vice President Quayle, Senator Mitchell, Speaker Wright, Senator Dole, Congressman Michael, and fellow citizens, neighbors, and friends: There is a man here who has earned a lasting place in our hearts and in our history. President Reagan, on behalf of our Nation, I thank you for the wonderful things that you have done for America. I have just repeated word for word the oath taken by George Washington 200 years ago, and the Bible on which I placed my hand is the Bible on which he placed his. It is right that the memory of Washington be with us today, not only because this is our Bicentennial Inauguration, but because Washington remains the Father of our Country. And he would, I think, be gladdened by this day; for today is the concrete expression of a stunning fact: our continuity these 200 years since our government began. We meet on democracy's front porch, a good place to talk as neighbors and as friends. For this is a day when our nation is made w

In [13]:
clinton_clean[:1000]

"My fellow citizens, today we celebrate the mystery of American renewal. This ceremony is held in the depth of winter, but by the words we speak and the faces we show the world, we force the spring. A spring reborn in the world's oldest democracy, that brings forth the vision and courage to reinvent America. When our founders boldly declared America's independence to the world, and our purposes to the Almighty, they knew that America, to endure, would have to change. Not change for change sake, but change to preserve America's ideals: life, liberty, the pursuit of happiness. Though we march to the music of our time, our mission is timeless. Each generation of American's must define what it means to be an American. On behalf of our nation, I salute my predecessor, President Bush, for his half-century of service to America and I thank the millions of men and women whose steadfastness and sacrifice triumphed over depression, fascism and communism. Today, a generation raised in the shadows

In [14]:
obama_clean[:1000]

'My fellow citizens: I stand here today humbled by the task before us, grateful for the trust you have bestowed, mindful of the sacrifices borne by our ancestors. I thank President Bush for his service to our nation, as well as the generosity and cooperation he has shown throughout this transition. Forty-four Americans have now taken the presidential oath. The words have been spoken during rising tides of prosperity and the still waters of peace. Yet, every so often the oath is taken amidst gathering clouds and raging storms. At these moments, America has carried on not simply because of the skill or vision of those in high office, but because We the People have remained faithful to the ideals of our forbearers, and true to our founding documents. So it has been. So it must be with this generation of Americans. That we are in the midst of crisis is now well understood. Our nation is at war, against a far-reaching network of violence and hatred. Our economy is badly weakened, a conseque

In [15]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
washington_doc = nlp(washington_clean)
jefferson_doc = nlp(jefferson_clean)
lincoln_doc = nlp(lincoln_clean)
roosevelt2_doc = nlp(roosevelt2_clean)
eisenhower_doc = nlp(eisenhower_clean)
kennedy_doc = nlp(kennedy_clean)
reagan_doc = nlp(reagan_clean)
bush_doc = nlp(bush_clean)
clinton_doc = nlp(clinton_clean)
obama_doc = nlp(obama_clean)

In [16]:
# Group into sentences.
washington_sents = [[sent, "Washington"] for sent in washington_doc.sents]
jefferson_sents = [[sent, "Jefferson"] for sent in jefferson_doc.sents]
lincoln_sents = [[sent, "Lincoln"] for sent in lincoln_doc.sents]
roosevelt2_sents = [[sent, "Roosevelt"] for sent in roosevelt2_doc.sents]
eisenhower_sents = [[sent, "Eisenhower"] for sent in eisenhower_doc.sents]
kennedy_sents = [[sent, "Kennedy"] for sent in kennedy_doc.sents]
reagan_sents = [[sent, "Reagan"] for sent in reagan_doc.sents]
bush_sents = [[sent, "Bush"] for sent in bush_doc.sents]
clinton_sents = [[sent, "Clinton"] for sent in clinton_doc.sents]
obama_sents = [[sent, "Obama"] for sent in obama_doc.sents]

In [17]:
# Combine the sentences into one data frame.
sentences = pd.DataFrame(washington_sents +
                         jefferson_sents +
                         lincoln_sents +
                         roosevelt2_sents +
                         eisenhower_sents +
                         kennedy_sents +
                         reagan_sents +
                         bush_sents +
                         clinton_sents +
                         obama_sents
                         )
sentences.head()

,0,1
0,"(Fellow, -, Citizens, of, the, Senate, and, of...",Washington
1,"(Among, the, vicissitudes, incident, to, life,...",Washington
2,"(On, the, one, hand, ,, I, was, summoned, by, ...",Washington
3,"(On, the, other, hand, ,, the, magnitude, and,...",Washington
4,"(In, this, conflict, of, emotions, all, I, dar...",Washington


In [18]:
# Set up bag of words function for each text.
def bag_of_words(text):
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(100)]


def bow_features(sentences, common_words):
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0

    for i, sentence in enumerate(df['text_sentence']):
        words = [token.lemma_
                 for token in sentence
                 if(
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        for word in words:
            df.loc[i, word] += 1
        if i % 100 == 0:
            print('Processing row {}'.format(i))
    return df

In [19]:
# Set up bags for each speech.
washington_words = bag_of_words(washington_doc)
jefferson_words = bag_of_words(jefferson_doc)
lincoln_words = bag_of_words(lincoln_doc)
roosevelt2_words = bag_of_words(roosevelt2_doc)
eisenhower_words = bag_of_words(eisenhower_doc)
kennedy_words = bag_of_words(kennedy_doc)
reagan_words = bag_of_words(reagan_doc)
bush_words = bag_of_words(bush_doc)
clinton_words = bag_of_words(clinton_doc)
obama_words = bag_of_words(obama_doc)

# Make bag of common words.
common_words = set(washington_words +
                   jefferson_words +
                   lincoln_words +
                   roosevelt2_words +
                   eisenhower_words +
                   kennedy_words +
                   reagan_words +
                   bush_words +
                   clinton_words +
                   obama_words
                   )

In [20]:
# Set up features from BoW.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800


,policy,burden,at,small,in,senator,freedom,celebrate,trial,when,...,tell,not,produce,bring,blessing,economy,sake,world,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Fellow, -, Citizens, of, the, Senate, and, of...",Washington
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Among, the, vicissitudes, incident, to, life,...",Washington
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(On, the, one, hand, ,, I, was, summoned, by, ...",Washington
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(On, the, other, hand, ,, the, magnitude, and,...",Washington
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, this, conflict, of, emotions, all, I, dar...",Washington


In [21]:
# See shape of word_counts.
word_counts.shape

(839, 459)

In [22]:
# Re-group into text sentences for tfidf.
washington_sents_text = [[sent.text, "Washington"]
                         for sent in washington_doc.sents]
jefferson_sents_text = [[sent.text, "Jefferson"]
                        for sent in jefferson_doc.sents]
lincoln_sents_text = [[sent.text, "Lincoln"] for sent in lincoln_doc.sents]
roosevelt2_sents_text = [[sent.text, "Roosevelt"]
                         for sent in roosevelt2_doc.sents]
eisenhower_sents_text = [[sent.text, "Eisenhower"]
                         for sent in eisenhower_doc.sents]
kennedy_sents_text = [[sent.text, "Kennedy"] for sent in kennedy_doc.sents]
reagan_sents_text = [[sent.text, "Reagan"] for sent in reagan_doc.sents]
bush_sents_text = [[sent.text, "Bush"] for sent in bush_doc.sents]
clinton_sents_text = [[sent.text, "Clinton"] for sent in clinton_doc.sents]
obama_sents_text = [[sent.text, "Obama"] for sent in obama_doc.sents]

In [23]:
# Combine the sentences from the speeches into one data frame.
sentences_text = pd.DataFrame(washington_sents_text +
                              jefferson_sents_text +
                              lincoln_sents_text +
                              roosevelt2_sents_text +
                              eisenhower_sents_text +
                              kennedy_sents_text +
                              reagan_sents_text +
                              bush_sents_text +
                              clinton_sents_text +
                              obama_sents_text
                              )
sentences_text.head()

,0,1
0,Fellow-Citizens of the Senate and of the House...,Washington
1,Among the vicissitudes incident to life no eve...,Washington
2,"On the one hand, I was summoned by my Country,...",Washington
3,"On the other hand, the magnitude and difficult...",Washington
4,In this conflict of emotions all I dare aver i...,Washington


In [24]:
sentences_text.shape

(839, 2)

In [25]:
# Append word_counts and sentences_text.
word_counts.append(sentences_text)
word_counts.head()

,policy,burden,at,small,in,senator,freedom,celebrate,trial,when,...,tell,not,produce,bring,blessing,economy,sake,world,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Fellow, -, Citizens, of, the, Senate, and, of...",Washington
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Among, the, vicissitudes, incident, to, life,...",Washington
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(On, the, one, hand, ,, I, was, summoned, by, ...",Washington
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(On, the, other, hand, ,, the, magnitude, and,...",Washington
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, this, conflict, of, emotions, all, I, dar...",Washington


In [27]:
# Clustering.
# Import clustering environment.
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Make sure the number of rows divides evenly into four samples.
rows = word_counts.shape[0] - word_counts.shape[0] % 4
word_counts = word_counts.iloc[:rows, :]

# Break into a set of features and a variable for the known outcome.
X = word_counts.iloc[:, :-2]
y = word_counts.iloc[:, -1]

# Normalize.
X_norm = normalize(X)

# Data frame to store features and predicted cluster memberships.
ypred = pd.DataFrame()

# Create the two-feature PCA for graphing purposes.
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_norm)

# Split the data into four equally-sized samples. First we break it in half:
X_half1, X_half2, X_pcahalf1, X_pcahalf2 = train_test_split(
    X_norm,
    X_pca,
    test_size=0.25,
    random_state=15)

# Then we half the halves.
X1, X2, X_pca1, X_pca2 = train_test_split(
    X_half1,
    X_pcahalf1,
    test_size=0.25,
    random_state=15)
X3, X4, X_pca3, X_pca4 = train_test_split(
    X_half2,
    X_pcahalf2,
    test_size=0.25,
    random_state=15)

In [33]:
# Pass a list of tuples and a counter that increments each time we go
# through the loop. The tuples are the data to be used by k-means,
# and the PCA-derived features for graphing. We use k-means to fit a
# model to the data, then store the predicted values and the two-feature
# PCA solution in the data frame.
for counter, data in enumerate([
    (X1, X_pca1),
    (X2, X_pca2),
    (X3, X_pca3),
    (X4, X_pca4)
]):

    # Put the features into ypred.
    ypred['pca_f1' + '_sample' + str(counter)] = data[1][:, 0]
    ypred['pca_f2' + '_sample' + str(counter)] = data[1][:, 1]

    # Generate cluster predictions and store them for clusters 2 to 6.
    for nclust in range(2, 7):
        pred = KMeans(n_clusters=nclust, random_state=15).fit_predict(data[0])
        ypred['clust' + str(nclust) + '_sample' + str(counter)] = pred

ValueError: Length of values does not match length of index

In [28]:
ypred.shape

(0, 0)

In [29]:
ypred.head()

""


In [30]:
pred.shape

(470,)

In [31]:
len(data)

2

In [32]:
data

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.33144851, -0.00186973],
        [-0.21058511, -0.10580197],
        [-0.16988775, -0.02700084],
        [-0.20965649,  0.10193564],
        [-0.20466654,  0.06373891],
        [-0.16968517, -0.03327104],
        [-0.19070935, -0.11257718],
        [-0.21109759,  0.2735665 ],
        [ 0.06043504, -0.00535084],
        [-0.18798858, -0.09336119],
        [ 0.51424234,  0.05137818],
        [-0.18974738, -0.06838806],
        [ 0.19756822, -0.06425613],
        [-0.16571706, -0.02814263],
        [-0.19148442, -0.05401993],
        [ 0.53055797,  0.01384743],
        [-0.21500285, -0.01714371],
        [-0.20266503,  0.16106598],
        [-0.18221862, -0.06492401],
        [-0.19654174, -0.16047944],
        [-0.1667961 , -0.0059692